In [33]:
import torch

In [1]:
!wget https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz

--2020-10-15 18:48:56--  https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.43.14
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.43.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442653086 (422M) [application/x-gzip]
Saving to: ‘amazon_reviews_us_Camera_v1_00.tsv.gz’

amazon_reviews_us_C 100%[===================>] 422.15M  73.8MB/s    in 5.6s    

2020-10-15 18:49:01 (75.6 MB/s) - ‘amazon_reviews_us_Camera_v1_00.tsv.gz’ saved [442653086/442653086]



In [1]:
import pandas as pd
df = pd.read_csv("amazon_reviews_us_Camera_v1_00.tsv.gz",  sep='\t', compression='gzip', error_bad_lines=False, dtype='str')

b'Skipping line 85458: expected 15 fields, saw 22\nSkipping line 91161: expected 15 fields, saw 22\n'
b'Skipping line 166123: expected 15 fields, saw 22\n'
b'Skipping line 225458: expected 15 fields, saw 22\nSkipping line 229936: expected 15 fields, saw 22\nSkipping line 259297: expected 15 fields, saw 22\n'
b'Skipping line 284728: expected 15 fields, saw 22\nSkipping line 286334: expected 15 fields, saw 22\nSkipping line 293400: expected 15 fields, saw 22\nSkipping line 294415: expected 15 fields, saw 22\nSkipping line 308150: expected 15 fields, saw 22\nSkipping line 315022: expected 15 fields, saw 22\nSkipping line 315730: expected 15 fields, saw 22\nSkipping line 316071: expected 15 fields, saw 22\nSkipping line 326729: expected 15 fields, saw 22\n'
b'Skipping line 329101: expected 15 fields, saw 22\nSkipping line 333077: expected 15 fields, saw 22\nSkipping line 377031: expected 15 fields, saw 22\nSkipping line 389496: expected 15 fields, saw 22\nSkipping line 390486: expected 15 

In [2]:
df.dropna(inplace=True)
df = df[ (df["review_body"].astype(str).str.len() >10) & (df["review_body"].astype(str).str.len() <=64)]
df = df[ (df["star_rating"] == "1") | (df["star_rating"] == "5")]
df = df[["review_body","star_rating"]]
df["star_rating"] =  df["star_rating"].astype(int)
df["star_rating"] =  df["star_rating"].apply(lambda x: 1 if x == 5 else 0)
df.columns = ["review","rating"]
# df = df[:100_000]
df.head()

,review,rating
1,"Perfect, even sturdier than the original!",1
7,Great camera for the price.,1
11,Use these all the time.,1
15,"very good on batteries, recommended",1
17,It is a very good starter set for this camera,1


In [3]:
from sklearn.utils import resample
df_majority = df[df['rating']==1]
df_minority = df[df['rating']==0]

maj_class1 = resample(df_majority, replace=True,n_samples=len(df_minority))
df = pd.concat([maj_class1, df_minority])
df["rating"].value_counts()

1    15551
0    15551
Name: rating, dtype: int64

In [6]:
max_len = 0
for sent in df["review"]:
    if len(sent) > max_len:
        max_len = len(sent)

In [7]:
max_len

64

In [8]:
sentences = df["review"].values
labels = df["rating"].values

In [9]:
df["rating"].value_counts()

1    15551
0    15551
Name: rating, dtype: int64

In [10]:
len(labels)

31102

In [11]:
#BERT
import transformers
from transformers import BertModel, BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import torch

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

from torch import nn, optim
from torch.utils.data import Dataset, DataLoader

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)


In [12]:
input_ids = []
attention_masks = []

for sent in sentences:
    encoded_dict = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 64,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        truncation=True,
                        return_tensors = 'pt'
                   )
    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

In [13]:
from torch.utils.data import TensorDataset, random_split

dataset = TensorDataset(input_ids, attention_masks, labels)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

27,991 training samples
3,111 validation samples


In [14]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 32

train_dataloader = DataLoader(
            train_dataset,
            sampler = RandomSampler(train_dataset),
            batch_size = batch_size
        )

validation_dataloader = DataLoader(
            val_dataset,
            sampler = SequentialSampler(val_dataset),
            batch_size = batch_size
        )

In [15]:
EPOCHS = 3
LEARNING_RATE = 1e-5

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def ret_model():
    model = BertForSequenceClassification.from_pretrained(
        "bert-base-uncased", 
        num_labels = 2, 
        output_attentions = False, 
        output_hidden_states = False,
    )
    return model

total_steps = len(train_dataloader) * EPOCHS

model = ret_model()
model.to(device)
optim = AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8)
scheduler = get_linear_schedule_with_warmup(optim, num_warmup_steps=0, num_training_steps=total_steps)


for epoch in range(0, EPOCHS):
    print("running epoch ",epoch)
   
    total_loss = 0
    model.train()
    for step, batch in enumerate(train_dataloader):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()
        loss, logits = model(b_input_ids, 
                        token_type_ids=None,
                        attention_mask = b_input_mask,
                        labels = b_labels)
        
        total_loss+= loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)


        optim.step()
        scheduler.step()
        
    avg_train_loss = total_loss / len(train_dataloader)
    print("avg_train_loss",avg_train_loss)
    
    model.eval()
    
    total_eval_acc = 0
    total_eval_loss = 0
    eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():
            loss, logits = model(b_input_ids, 
                        token_type_ids=None,
                        attention_mask = b_input_mask,
                        labels = b_labels)

        total_eval_loss+= loss.item()
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to("cpu").numpy()
        
        total_eval_acc += flat_accuracy(logits, label_ids)


    avg_val_acc = total_eval_acc / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_acc))
    
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))


running epoch  0


/opt/conda/conda-bld/pytorch_1587428091666/work/torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha)


avg_train_loss 0.12130771941159453
  Accuracy: 0.98
  Validation Loss: 0.08
running epoch  1
avg_train_loss 0.06251509219620908
  Accuracy: 0.98
  Validation Loss: 0.07
running epoch  2
avg_train_loss 0.046090476284069676
  Accuracy: 0.98
  Validation Loss: 0.08


In [16]:
torch.save(model.state_dict(), "model.pth")

In [17]:
model = ret_model()
model.load_state_dict(torch.load("model.pth"))
model.eval()
print("")

In [18]:
def detect_sentiment(sentence):
    inputs = tokenizer.encode_plus(
                sentence,
                add_special_tokens=True,
                max_length=64,
                return_token_type_ids=True,
                pad_to_max_length=True,
                return_attention_mask=True,
                return_tensors='pt'
            )
    ids = inputs["input_ids"]
    mask = inputs["attention_mask"]
    token_type_ids = inputs["token_type_ids"]

    outputs = model(ids, token_type_ids=token_type_ids, attention_mask=mask)
    preds = torch.sigmoid(outputs[0]).cpu().detach().numpy()
    return np.argmax(preds, axis=1)

In [19]:
detect_sentiment("this is great")

array([1])

In [20]:
detect_sentiment("worst product ever")

array([0])